In [1]:
import requests
import pandas as pd
import numpy as np
ua = {#'User-Agent':'Mozilla/5.0 (compatible; Googlebot/2.1; +Googlebot - Webmaster Tools Help)',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.124 Safari/537.36',
'Connection':'Keep-Alive',
'Accept-Language':'zh-CN,zh;q=0.8',
'Accept-Encoding':'gzip,deflate,sdch',
'Accept':'*/*',
'Accept-Charset':'GBK,utf-8;q=0.7,*;q=0.3',
'Cache-Control':'max-age=0'
}
def financial_statement(year, season, type='綜合損益彙總表'):

    if year >= 1000:
        year -= 1911
        
    if type == '綜合損益彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == '資產負債彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif type == '營益分析彙總表':
        url = 'http://mops.twse.com.tw/mops/web/ajax_t163sb06'
    else:
        print('type does not match')

    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':'sii',   
        'year':str(year),
        'season':str(season),
    })
    
    r.encoding = 'utf8'
    dfs = pd.read_html(r.text)
    
    
    for i, df in enumerate(dfs):
        df.columns = df.iloc[0]
        dfs[i] = df.iloc[1:]
        
    df = pd.concat(dfs).applymap(lambda x: x if x != '--' else np.nan)
    df = df[df['公司代號'] != '公司代號']
    df = df[~df['公司代號'].isnull()]
    return df

In [2]:
df=financial_statement(2019, 1,'營益分析彙總表')
df1=financial_statement(2018,4,'營益分析彙總表')

C:\Users\ASUS\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [3]:
df.head()

,公司代號,公司名稱,合計：共 897 家,毛利率(%)(營業毛利)/(營業收入),營業利益率(%)(營業利益)/(營業收入),營業收入(百萬元),稅前純益率(%)(稅前純益)/(營業收入),稅後純益率(%)(稅後純益)/(營業收入)
1,1101,台泥,NaN,25.59,19.85,25356.33,22.10,16.68
2,1102,亞泥,NaN,24.03,19.49,19385.68,24.47,19.65
3,1103,嘉泥,NaN,14.42,-12.37,452.38,37.91,32.84
4,1104,環球水泥,NaN,8.96,0.85,1187.30,13.25,13.19
5,1108,幸福水泥,NaN,3.20,-3.16,834.35,-4.20,-3.57


In [4]:
df1.head()

,公司代號,公司名稱,合計：共 898 家,毛利率(%)(營業毛利)/(營業收入),營業利益率(%)(營業利益)/(營業收入),營業收入(百萬元),稅前純益率(%)(稅前純益)/(營業收入),稅後純益率(%)(稅後純益)/(營業收入)
1,1101,台泥,NaN,26.96,22.62,124594.60,24.52,18.17
2,1102,亞泥,NaN,25.59,21.94,82741.00,24.62,17.99
3,1103,嘉泥,NaN,13.79,-4.95,2092.41,20.45,36.76
4,1104,環球水泥,NaN,11.94,3.82,4780.99,23.59,21.99
5,1108,幸福水泥,NaN,1.40,-4.66,3330.60,-5.75,-6.05


In [23]:
df['季'] = 201901
df1['季'] = 201804

df = df.drop(['合計：共 897 家'], axis=1)
df1 = df1.drop(['合計：共 899 家'], axis=1)

KeyError: "['合計：共 899 家'] not found in axis"

In [24]:
df.head()

,公司代號,公司名稱,毛利率(%)(營業毛利)/(營業收入),營業利益率(%)(營業利益)/(營業收入),營業收入(百萬元),稅前純益率(%)(稅前純益)/(營業收入),稅後純益率(%)(稅後純益)/(營業收入),季
1,1101,台泥,25.59,19.85,25356.33,22.10,16.68,201901
2,1102,亞泥,24.03,19.49,19385.68,24.47,19.65,201901
3,1103,嘉泥,14.42,-12.37,452.38,37.91,32.84,201901
4,1104,環球水泥,8.96,0.85,1187.30,13.25,13.19,201901
5,1108,幸福水泥,3.20,-3.16,834.35,-4.20,-3.57,201901


In [25]:
df_2s = pd.merge(df, df1, how='outer' ,on='公司代號')

In [26]:
df_2s

,公司代號,公司名稱_x,毛利率(%)(營業毛利)/(營業收入)_x,營業利益率(%)(營業利益)/(營業收入)_x,營業收入(百萬元)_x,稅前純益率(%)(稅前純益)/(營業收入)_x,稅後純益率(%)(稅後純益)/(營業收入)_x,季_x,公司名稱_y,合計：共 898 家,毛利率(%)(營業毛利)/(營業收入)_y,營業利益率(%)(營業利益)/(營業收入)_y,營業收入(百萬元)_y,稅前純益率(%)(稅前純益)/(營業收入)_y,稅後純益率(%)(稅後純益)/(營業收入)_y,季_y
0,1101,台泥,25.59,19.85,25356.33,22.10,16.68,201901.0,台泥,NaN,26.96,22.62,124594.60,24.52,18.17,201804
1,1102,亞泥,24.03,19.49,19385.68,24.47,19.65,201901.0,亞泥,NaN,25.59,21.94,82741.00,24.62,17.99,201804
2,1103,嘉泥,14.42,-12.37,452.38,37.91,32.84,201901.0,嘉泥,NaN,13.79,-4.95,2092.41,20.45,36.76,201804
3,1104,環球水泥,8.96,0.85,1187.30,13.25,13.19,201901.0,環球水泥,NaN,11.94,3.82,4780.99,23.59,21.99,201804
4,1108,幸福水泥,3.20,-3.16,834.35,-4.20,-3.57,201901.0,幸福水泥,NaN,1.40,-4.66,3330.60,-5.75,-6.05,201804
5,1109,信大水泥,28.62,20.90,1360.43,20.60,14.89,201901.0,信大水泥,NaN,32.59,26.19,6701.13,26.25,18.71,201804
6,1110,東泥,8.38,3.51,417.04,7.97,7.15,201901.0,東泥,NaN,8.62,2.42,1676.19,4.25,3.91,201804
7,1201,味全公司,29.73,-0.32,4540.80,28.38,24.65,201901.0,味全公司,NaN,31.07,2.64,19436.76,5.11,4.47,201804
8,1203,味王公司,31.21,14.77,1486.29,15.40,12.15,201901.0,味王公司,NaN,32.40,16.06,6081.03,16.77,11.58,201804
9,1210,大成長城,14.60,3.71,18186.19,3.82,2.75,201901.0,大成長城,NaN,13.93,3.14,72442.93,4.18,3.33,201804


In [27]:
df_2s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 898 entries, 0 to 897
Data columns (total 16 columns):
公司代號                       898 non-null object
公司名稱_x                     897 non-null object
毛利率(%)(營業毛利)/(營業收入)_x      896 non-null object
營業利益率(%)(營業利益)/(營業收入)_x    896 non-null object
營業收入(百萬元)_x                896 non-null object
稅前純益率(%)(稅前純益)/(營業收入)_x    896 non-null object
稅後純益率(%)(稅後純益)/(營業收入)_x    896 non-null object
季_x                        897 non-null float64
公司名稱_y                     898 non-null object
合計：共 898 家                 0 non-null object
毛利率(%)(營業毛利)/(營業收入)_y      898 non-null object
營業利益率(%)(營業利益)/(營業收入)_y    898 non-null object
營業收入(百萬元)_y                898 non-null object
稅前純益率(%)(稅前純益)/(營業收入)_y    898 non-null object
稅後純益率(%)(稅後純益)/(營業收入)_y    898 non-null object
季_y                        898 non-null int64
dtypes: float64(1), int64(1), object(14)
memory usage: 119.3+ KB


In [28]:
df_2s['稅後純益率(%)(稅後純益)/(營業收入)_x'] = df_2s['稅後純益率(%)(稅後純益)/(營業收入)_x'].astype('float32')
df_2s['稅後純益率(%)(稅後純益)/(營業收入)_y'] = df_2s['稅後純益率(%)(稅後純益)/(營業收入)_y'].astype('float32')
df_2s['公司代號'] = df_2s['公司代號'].astype('int32')
df_2s['毛利率(%)(營業毛利)/(營業收入)_x'] = df_2s['毛利率(%)(營業毛利)/(營業收入)_x'].astype('float32')
df_2s['毛利率(%)(營業毛利)/(營業收入)_y'] = df_2s['毛利率(%)(營業毛利)/(營業收入)_y'].astype('float32')
df_2s['營業利益率(%)(營業利益)/(營業收入)_x'] = df_2s['營業利益率(%)(營業利益)/(營業收入)_x'].astype('float32')
df_2s['營業利益率(%)(營業利益)/(營業收入)_y'] = df_2s['營業利益率(%)(營業利益)/(營業收入)_y'].astype('float32')
df_2s['稅前純益率(%)(稅前純益)/(營業收入)_x'] = df_2s['稅前純益率(%)(稅前純益)/(營業收入)_y'].astype('float32')
df_2s['稅前純益率(%)(稅前純益)/(營業收入)_y'] = df_2s['稅前純益率(%)(稅前純益)/(營業收入)_y'].astype('float32')
df_2s['稅後純益率差'] = df_2s['稅後純益率(%)(稅後純益)/(營業收入)_x'] - df_2s['稅後純益率(%)(稅後純益)/(營業收入)_y']

In [29]:
df3 = df_2s.loc[df_2s.稅後純益率差>0]
df3

,公司代號,公司名稱_x,毛利率(%)(營業毛利)/(營業收入)_x,營業利益率(%)(營業利益)/(營業收入)_x,營業收入(百萬元)_x,稅前純益率(%)(稅前純益)/(營業收入)_x,稅後純益率(%)(稅後純益)/(營業收入)_x,季_x,公司名稱_y,合計：共 898 家,毛利率(%)(營業毛利)/(營業收入)_y,營業利益率(%)(營業利益)/(營業收入)_y,營業收入(百萬元)_y,稅前純益率(%)(稅前純益)/(營業收入)_y,稅後純益率(%)(稅後純益)/(營業收入)_y,季_y,稅後純益率差
1,1102,亞泥,24.030001,19.490000,19385.68,24.620001,19.650000,201901.0,亞泥,NaN,25.590000,21.940001,82741.00,24.620001,17.990000,201804,1.660000
4,1108,幸福水泥,3.200000,-3.160000,834.35,-5.750000,-3.570000,201901.0,幸福水泥,NaN,1.400000,-4.660000,3330.60,-5.750000,-6.050000,201804,2.480000
6,1110,東泥,8.380000,3.510000,417.04,4.250000,7.150000,201901.0,東泥,NaN,8.620000,2.420000,1676.19,4.250000,3.910000,201804,3.240000
7,1201,味全公司,29.730000,-0.320000,4540.80,5.110000,24.650000,201901.0,味全公司,NaN,31.070000,2.640000,19436.76,5.110000,4.470000,201804,20.180000
8,1203,味王公司,31.209999,14.770000,1486.29,16.770000,12.150000,201901.0,味王公司,NaN,32.400002,16.059999,6081.03,16.770000,11.580000,201804,0.570000
10,1215,卜蜂企業,14.350000,6.700000,5035.30,6.160000,4.950000,201901.0,卜蜂企業,NaN,13.460000,6.050000,21235.09,6.160000,4.690000,201804,0.260000
11,1216,統一,34.419998,6.970000,107791.64,8.180000,7.260000,201901.0,統一,NaN,33.669998,6.250000,431445.52,8.180000,6.250000,201804,1.010000
12,1217,愛之味,30.920000,2.320000,996.83,1.350000,1.660000,201901.0,愛之味,NaN,31.940001,1.550000,4321.82,1.350000,1.450000,201804,0.210000
13,1218,泰山企業,18.139999,1.980000,1731.21,6.220000,6.100000,201901.0,泰山企業,NaN,18.120001,1.140000,7901.67,6.220000,5.880000,201804,0.220000
14,1219,福壽實業,8.880000,0.640000,2966.59,-0.720000,0.510000,201901.0,福壽實業,NaN,6.750000,-0.710000,11709.98,-0.720000,-0.820000,201804,1.330000


In [30]:
df3.sort_values(by='稅後純益率差', ascending=False)

,公司代號,公司名稱_x,毛利率(%)(營業毛利)/(營業收入)_x,營業利益率(%)(營業利益)/(營業收入)_x,營業收入(百萬元)_x,稅前純益率(%)(稅前純益)/(營業收入)_x,稅後純益率(%)(稅後純益)/(營業收入)_x,季_x,公司名稱_y,合計：共 898 家,毛利率(%)(營業毛利)/(營業收入)_y,營業利益率(%)(營業利益)/(營業收入)_y,營業收入(百萬元)_y,稅前純益率(%)(稅前純益)/(營業收入)_y,稅後純益率(%)(稅後純益)/(營業收入)_y,季_y,稅後純益率差
65,1438,裕豐,-760.000000,-46460.000000,0.01,-69876.468750,23060.000000,201901.0,裕豐,NaN,-211.759995,-48211.761719,0.02,-69876.468750,-69876.468750,201804,92936.468750
78,1453,大將,57.380001,-2961.600098,0.24,1187.750000,4583.120117,201901.0,大將,NaN,-1.690000,-144.679993,28.28,1187.750000,1185.550049,201804,3397.570068
470,2923,鼎固-KY,64.629997,22.000000,1159.81,45.849998,712.090027,201901.0,鼎固-KY,NaN,57.709999,60.220001,6928.72,45.849998,23.570000,201804,688.520020
87,1465,偉全實業,12.880000,-41.580002,199.92,4.070000,500.299988,201901.0,偉全實業,NaN,12.990000,0.340000,697.15,4.070000,-7.860000,201804,508.159973
62,1435,中福國際,94.080002,-78.830002,5.35,-40.560001,237.210007,201901.0,中福國際,NaN,94.389999,-25.490000,30.14,-40.560001,-40.759998,201804,277.970001
596,3686,達能,-21.059999,-60.990002,32.28,-352.570007,-117.660004,201901.0,達能,NaN,-152.369995,-163.809998,522.53,-352.570007,-353.429993,201804,235.769989
407,2538,基泰建設,71.139999,-15.520000,35.35,36.570000,186.559998,201901.0,基泰建設,NaN,69.580002,-10.070000,148.33,36.570000,34.080002,201804,152.479996
714,6131,悠克國際,125.970001,257.970001,-8.86,167.270004,274.950012,201901.0,悠克國際,NaN,105.570000,132.710007,-488.06,167.270004,167.279999,201804,107.670013
469,2915,潤泰全球,21.760000,-5.820000,870.22,123.070000,242.410004,201901.0,潤泰全球,NaN,59.730000,46.230000,10498.33,123.070000,146.850006,201804,95.559998
46,1337,再生-KY,-25.459999,-54.230000,374.81,-128.270004,-52.430000,201901.0,再生-KY,NaN,-53.000000,-124.269997,1464.80,-128.270004,-128.919998,201804,76.489998


In [31]:
df4 = df_2s.loc[(df_2s["毛利率(%)(營業毛利)/(營業收入)_x"] - df_2s["毛利率(%)(營業毛利)/(營業收入)_y"]>=0) &
                (df_2s["營業利益率(%)(營業利益)/(營業收入)_x"] - df_2s["營業利益率(%)(營業利益)/(營業收入)_y"]>=0) &
                (df_2s["稅前純益率(%)(稅前純益)/(營業收入)_x"] - df_2s["稅前純益率(%)(稅前純益)/(營業收入)_y"]>=0) &
                (df_2s["稅後純益率(%)(稅後純益)/(營業收入)_x"] - df_2s["稅後純益率(%)(稅後純益)/(營業收入)_y"]>=0)
               ]

In [32]:
df5 =df4.sort_values(by='稅後純益率差', ascending=False)

In [33]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204 entries, 4 to 896
Data columns (total 17 columns):
公司代號                       204 non-null int32
公司名稱_x                     204 non-null object
毛利率(%)(營業毛利)/(營業收入)_x      204 non-null float32
營業利益率(%)(營業利益)/(營業收入)_x    204 non-null float32
營業收入(百萬元)_x                204 non-null object
稅前純益率(%)(稅前純益)/(營業收入)_x    204 non-null float32
稅後純益率(%)(稅後純益)/(營業收入)_x    204 non-null float32
季_x                        204 non-null float64
公司名稱_y                     204 non-null object
合計：共 898 家                 0 non-null object
毛利率(%)(營業毛利)/(營業收入)_y      204 non-null float32
營業利益率(%)(營業利益)/(營業收入)_y    204 non-null float32
營業收入(百萬元)_y                204 non-null object
稅前純益率(%)(稅前純益)/(營業收入)_y    204 non-null float32
稅後純益率(%)(稅後純益)/(營業收入)_y    204 non-null float32
季_y                        204 non-null int64
稅後純益率差                     204 non-null float32
dtypes: float32(9), float64(1), int32(1), int64(1), object(5)
memory usage: 20.7+ KB


In [34]:
import os
os.chdir("C:/Users/ASUS/Desktop/股票學習資料夾/2019Q1")   # 指定路徑(預設會存在程式碼所在路徑下)
df5.to_excel('20190516.xlsx',    encoding='utf_8_sig',    index = False  )

In [35]:
df_2s.loc[df_2s.公司代號 == 4943]

,公司代號,公司名稱_x,毛利率(%)(營業毛利)/(營業收入)_x,營業利益率(%)(營業利益)/(營業收入)_x,營業收入(百萬元)_x,稅前純益率(%)(稅前純益)/(營業收入)_x,稅後純益率(%)(稅後純益)/(營業收入)_x,季_x,公司名稱_y,合計：共 898 家,毛利率(%)(營業毛利)/(營業收入)_y,營業利益率(%)(營業利益)/(營業收入)_y,營業收入(百萬元)_y,稅前純益率(%)(稅前純益)/(營業收入)_y,稅後純益率(%)(稅後純益)/(營業收入)_y,季_y,稅後純益率差
661,4943,康控-KY,14.66,-16.48,591.35,25.77,-26.4,201901.0,康控-KY,NaN,41.150002,25.879999,5757.90,25.77,16.959999,201804,-43.360001
